In [0]:

%run ./01-config

In [0]:
class Upserter:
    def __init__(self, merge_query, temp_view):
        self.merge_query = merge_query
        self.temp_view = temp_view

    def upsert(self, df_micro_batch, batch_id):
        df_micro_batch.createOrReplaceTempView(self.temp_view)
        df_microbatch.sparkSession.sql(self.merge_query)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql import functions as f

class CDCUpserter:
    def __init__(self, merge_query, temp_view, id_col, sort_col):
        self.merge_query = merge_query
        self.temp_view = temp_view
        self.id_col = id_col
        self.sort_col = sort_col

    def upsert(self, df_micro_batch, batch_id):
        window = Window.partition_by(self.id_col).orderBy(F.col(self.sort_col).desc())

        df_micro_batch.filter(F.col("row_status").isin(["insert", "update"])) \
            .withColumn("rank", F.rank().over(window)) \
            .filter("rank == 1") \
            .drop("rank") \
            .createOrReplaceTempView(self.temp_view)

        df_micro_batch.sparkSession.sql(self.merge_query)